Test

In [ ]:

!pip install bitarray fastBPE hydra-core omegaconf regex requests sacremoses subword_nmt sacrebleu


In [ ]:
import torch

# List available models
torch.hub.list('pytorch/fairseq')  # [..., 'transformer_lm.wmt19.en', ...]

In [ ]:
import torch

class Seer:
    def __init__(self, dataset = "roberta.base"):
        roberta = torch.hub.load("pytorch/fairseq", dataset)
        roberta.eval()
        self.model = roberta
        self.tail = ["?", "!", "»"]
        self.deadtail = [".", ":", "...", "…", ",", ";"]
        self.end = ["?", "!", "!)", '?"', "»", "?]", "[", "(", "?'", "???", "(?", "('"]
        self.deadend = [".", ":", "...", "…", ",", ";", "...?", "]", ")", "'?", "?)", "')"]

    def predict(self, text, i=0, sentence=""):
        predictions = self.model.fill_mask(text + " <mask>", topk=5)
        unmasked = predictions[i][2].strip()
        if len(sentence.split(" ")) < 4:
            if unmasked not in self.tail and unmasked not in self.deadtail:
                sentence += " " + unmasked
                text += " " + unmasked
                return(self.predict(text, 0, sentence))
        if len(sentence.split(" ")) == 1:
            return unmasked
        if unmasked in self.tail:
            return sentence.strip() + " " + unmasked
        if unmasked in self.deadtail:
            return sentence.strip() + unmasked

    def predict_further(self, text, i=0, sentence=""):
        length = len(sentence.split(" "))
        predictions = self.model.fill_mask(text + "<mask>", topk=10)
        unmasked = predictions[i][2].strip()
        if length < 4:
            if unmasked in self.end or unmasked in self.deadend:
                return self.predict_further(text, i + 1, sentence)
            sentence += " " + unmasked
            text += " " + unmasked
            return(self.predict_further(text, 0, sentence))
        if unmasked in self.end:
            return sentence.strip() + " " + unmasked
        if unmasked in self.deadend:
            return sentence.strip() + unmasked
        return sentence.strip() + " " + unmasked

    def predict_2(self, data):
        predict = []
        for i in range(5):
            text = data["text"].strip()
            result = self.predict(text, i)
            if result not in predict:
                predict.append(result)
        return predict

    def predict_4(self, data):
        predict = []
        for i in range(5):
            text = data["text"].strip()
            try:
                result = self.predict_further(text, i)
                if result not in predict:
                    predict.append(result)
            except:
                continue
        return predict

Nostradamus = Seer()



Using cache found in /root/.cache/torch/hub/pytorch_fairseq_main
2022-01-22 03:11:43 | INFO | fairseq.file_utils | loading archive file http://dl.fbaipublicfiles.com/fairseq/models/roberta.base.tar.gz from cache at /root/.cache/torch/pytorch_fairseq/37d2bc14cf6332d61ed5abeb579948e6054e46cc724c7d23426382d11a31b2d6.ae5852b4abc6bf762e0b6b30f19e741aa05562471e9eb8f4a6ae261f04f9b350
/usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:36: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  message="hydra.experimental.initialize() is no longer experimental."
/usr/local/lib/python3.7/dist-packages/hydra/experimental/compose.py:19: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  message="hydra.experimental.compose() is no longer experimental."
/usr/local/lib/python3.7/dist-packages/hydra/core/default_element.py:126: UserWarning: In 'config': Usage of deprecated keyword in package header '#

In [ ]:
req_data = "Michael Obama is "

content = Nostradamus.predict_2({
    "text": req_data
})
print(content)

['dead.', 'gay.', 'president.', 'President.', 'retiring.']
